In [1]:
import optuna
import joblib
import numpy as np
import pandas as pd
import sys
from concurrent.futures import ThreadPoolExecutor
from optuna.storages import JournalStorage, JournalFileStorage
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import load_ml_data, HyperparamTuner

importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [2]:
# sampler - a method used to generate new sets of hyperparameters in each iteration of the optimization process
samplers = {
    'RandomSampler': optuna.samplers.RandomSampler,          # Sampler that selects hyperparameters randomly from the search space.
    'GridSampler': optuna.samplers.GridSampler,              # Sampler that performs a grid search over the hyperparameter space.
    'TPESampler': optuna.samplers.TPESampler,                # Sampler that uses a tree-structured Parzen estimator to model the objective function and sample new points from the search space.
    'CmaEsSampler': optuna.samplers.CmaEsSampler,            # Sampler that uses the Covariance Matrix Adaptation Evolution Strategy algorithm to efficiently search the hyperparameter space.
    'NSGAIISampler': optuna.samplers.NSGAIISampler,          # Multi-objective evolutionary algorithm that generates new samples using non-dominated sorting and crowding distance selection.
    'QMCSampler': optuna.samplers.QMCSampler,                # Quasi-Monte Carlo sampler that uses low-discrepancy sequences to sample the search space in a more efficient and evenly distributed way than random sampling.
    'BoTorchSampler': optuna.integration.BoTorchSampler,     # Sampler that leverages the BoTorch library for Bayesian optimization and can handle both continuous and categorical hyperparameters.
    'BruteForceSampler': optuna.samplers.BruteForceSampler,  # Sampler that exhaustively evaluates all possible combinations of hyperparameters in the search space.
}
# pruner - a technique used to eliminate unpromising trials during the course of hyperparameter optimization.
pruners = {
    'BasePruner': optuna.pruners.BasePruner,                            # This is the base class for all pruning strategies in Optuna. It provides a skeleton for implementing custom pruning strategies.
    'MedianPruner': optuna.pruners.MedianPruner,                        # A pruner that prunes unpromising trials that have median objective values, as determined in previous steps.
    'SuccessiveHalvingPruner': optuna.pruners.SuccessiveHalvingPruner,  # This pruner repeatedly splits trials into halves, discarding the lower performing half at each iteration.
    'HyperbandPruner': optuna.pruners.HyperbandPruner,                  # This pruner implements the Hyperband algorithm, which selects promising trials and runs them with different resource allocation schemes to determine the best one.
    'PercentilePruner': optuna.pruners.PercentilePruner,                # A pruner that prunes unpromising trials based on their percentile rank relative to all completed trials.
    'NopPruner': optuna.pruners.NopPruner,                              # A pruner that does nothing and does not prune any trials.
    'ThresholdPruner': optuna.pruners.ThresholdPruner,                  # This pruner prunes trials that have not reached a certain level of performance (i.e., objective value).
    'PatientPruner': optuna.pruners.PatientPruner,                      # This pruner prunes trials that do not show improvement over a certain number of steps (or epochs).
}

In [ ]:
sampler = samplers['TPESampler']
pruner = pruners["HyperbandPruner"]
n_trials = 1000
with ThreadPoolExecutor() as executor:
    futures = []
    for _type in ["morgan", "jazzy"]:
        for benchmark in ["obach", "hepatocyte", "microsome"]:
            X_train, y_train, X_val, y_val, X_test, y_test = load_ml_data(_type, benchmark)

            for model_identifier in ["linear", "KRR", "GB", "RF", "ANN"]:
                print(_type, benchmark, model_identifier)

                lock_obj = optuna.storages.JournalFileOpenLock(
                    f"./project_resources/optuna/{_type}/{benchmark}/{model_identifier}_journal.log"
                )

                storage = JournalStorage(
                                JournalFileStorage(
                                    f"./project_resources/optuna/{_type}/{benchmark}/{model_identifier}_journal.log", lock_obj=lock_obj
                                )
                            )
    
                study = optuna.create_study(
                    study_name=model_identifier, direction="maximize", storage=storage, load_if_exists=True,
                    sampler=sampler(n_startup_trials=300, constant_liar=True), pruner=pruner()
                )
                
                trial_log_csv_path = f"./project_resources/optuna/{_type}/{benchmark}/{model_identifier}.csv"
                
                tuner = HyperparamTuner(trial_log_csv_path, model_identifier, X_train, y_train, X_val, y_val)
                
                futures.append(executor.submit(study.optimize, tuner.objective, n_trials=n_trials))
                joblib.dump(study, f"./project_resources/optuna/{_type}/{benchmark}/{model_identifier}.pkl")
    for future in futures:
        future.result()

Found local copy...
Loading...
Done!
100%|███████████████████████████████████████████████████████████████████████████████| 667/667 [00:01<00:00, 642.19it/s]
[I 2024-03-19 11:12:05,450] A new study created in Journal with name: linear
[I 2024-03-19 11:12:05,474] A new study created in Journal with name: KRR
[I 2024-03-19 11:12:05,507] A new study created in Journal with name: GB
[I 2024-03-19 11:12:05,548] A new study created in Journal with name: RF


morgan obach linear
morgan obach KRR
morgan obach GB
morgan obach RF


[I 2024-03-19 11:12:05,660] A new study created in Journal with name: ANN
Found local copy...
Loading...
Done!


morgan obach ANN


 80%|██████████████████████████████████████████████████████████████▍               | 971/1213 [00:01<00:00, 667.44it/s]

...

[I 2024-03-19 11:26:01,310] Trial 8 finished with value: 0.22107968542568046 and parameters: {'learning_rate_init': 0.11556082156914915, 'hidden_layer_sizes': [10, 10, 10]}. Best is trial 8 with value: 0.22107968542568046.
[I 2024-03-19 11:26:27,153] Trial 8 finished with value: 0.2149754062428326 and parameters: {'learning_rate_init': 0.11316206963853503, 'hidden_layer_sizes': [10, 10, 10]}. Best is trial 7 with value: 0.23875919120528677.


Successfully updated ANN.csv with results of trial 8
Successfully updated RF.csv with results of trial 8


[I 2024-03-19 11:26:29,097] Trial 8 finished with value: 0.3334759991873479 and parameters: {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': 20}. Best is trial 4 with value: 0.35035892191170576.
[I 2024-03-19 11:26:52,917] Trial 6 pruned. 


Successfully updated RF.csv with results of trial 6


[I 2024-03-19 11:26:59,692] Trial 6 finished with value: 0.3310448607640261 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 10}. Best is trial 5 with value: 0.3695008600106641.
